# Accounts Learning Item

### What is an account?

We have completed creating a class to represent a single position, but we often have multiple positions and want to organize and group them. This is what we call an account. An account is a logical group of positions that are related. For example, you may have an account specifically dedicated to your retirement goals. This account may hold positions and securities that will grow over time. You may also have an account for a trip you want to take next year which contains fixed income debt, which won't won't fluctuate rapidly in value. 

Below is an example of an account you could have.

I have an account named *"My Retirement Account"* which holds:

 ***2,000*** shares of ***MSFT USD***, 
 ***200*** shares of ***AAPL USD***, and
 ***100*** shares of ***TSLA USD***

### Problem Definition

We want to build a class that represents an account. We should be able to construct accounts with a set of positions and an account name. We should be able to get the name of the account from the object. 

We should be able to query with a set of security names/security objects (the list could contain both) and return a map of our input to the matching position object if present.
Additionally we should be able to add an arbitrary set of positions to our account, with duplicates being able to set existing positions. 

Lastly we should be able to remove accounts based on a set of incoming security names/security objects, any securities that aren't actually contained can be ignored.

- Allow for an account to be created with a set of positions and an account name
- Allow for querying of the current account's name
- Allow for the querying of all positions
- Allow for querying of positions with a set of security names/security objects
    - The set can contain both security name string and security objects
    - The return value should be a map with the query value to the position
    - The set can contain securities not present in the account. These should be ignored.
- Allow for positions to be added to the account with a set of position objects. Incoming positions should update existing positions
- Allow for the removal of positions from the account with a set of security names/security objects. Securities not in the account should be ignored.

### Provided Tools

#### *Data Source*

For this section no data generators are provided

#### *Solution Interface*

Your solution will need to follow the interface provided in the lab. Below is a snippet of the interface for securities that you can inherit from. The methods that will be tested are displayed & will need to be overwritten with your implementation. You're free to add more methods then what is displayed in the interface!

```python
#filename interfaces.account_interface.py
#Account Class Interface

from .position_interface import PositionInterface
from typing import Any, Dict, Set, Iterable
class AccountInterface():
    def __init__(self, positions: Set[positionInterface], account_name: str) -> None:
        pass

    def get_name(self) -> str:
        pass

    def get_all_positions(self) -> Iterable[PositionInterface]:
        pass

    def get_positions(self, securities: Set) -> Dict[Any, PositionInterface]:
        pass

    def add_positions(self, positions: Set[PositionInterface]) -> None:
        pass
    
    def remove_positions(self, securities: Set) -> None:
        pass
```

To successfully import the interface into your solution cell you'll need to add the below code snippet.

```python
from interfaces.account_interface import AccountInterface
```

#### *Testing*

Once you have completed & saved your solution you can run the test file to validate that your solution works as expected. For the test to run the following need to be true.
- Saved code to file **implementations/account_solution.py**
- Create a class with the name **Account** that inherits from **AccountInterface**

### Stretch Goals

If you complete your class & have a solution with valid tests try completing the following stretch goals 

- Update your class init to handle the position set being optional
- Add custom implementation for class's __str__ method. The account print method shouldn't reimplement the positions's print str method but should utilize it.
- Add a method to remove all positions from an account.
- Develop tests for the new methods created

In [ ]:
#%%writefile ../implementations/account_solution.py 
#Uncomment line above & run cell to save solution
#TODO Define class that implements accountInterface & allows for the management of an account

In [ ]:
#%conda install ipytest
#%pip install ipytest
import ipytest

ipytest.autoconfig()

In [ ]:
%%ipytest -qq


import implementations.account_solution
from implementations.position_solution import Position
from implementations.security_solution import Security

import importlib

importlib.reload(implementations.account_solution)


def test_get_account_name():
    # GIVEN
    expected_name = "MY TEST ACCOUNT"
    expected_positions = set()

    # WHEN
    test_obj = implementations.account_solution.Account(
        expected_positions, expected_name
    )

    # EXPECT
    assert test_obj.get_name() == expected_name


def test_get_all_positions():
    # GIVEN
    expected_name = "MY TEST ACCOUNT"
    expected_positions = set()
    expected_positions.add(Position("TEST_SEC_A", 1000))
    expected_positions.add(Position("TEST_SEC_B", 2000))

    # WHEN
    test_obj = implementations.account_solution.Account(
        expected_positions, expected_name
    )
    return_pos_itr = test_obj.get_all_positions()

    # EXPECT
    assert len(return_pos_itr) == len(expected_positions)

    for item in list(return_pos_itr):
        assert item in expected_positions
        expected_positions.remove(item)
        return_pos_itr.remove(item)

    assert len(return_pos_itr) == 0
    assert len(expected_positions) == 0


def test_get_positions():
    # GIVEN
    expected_name = "MY TEST ACCOUNT"
    expected_positions = set()
    expected_positions.add(Position("TEST_SEC_A", 1000))
    expected_positions.add(Position("TEST_SEC_B", 2000))
    key_list = [
        Security("TEST_SEC_A"),
        "TEST_SEC_B",
        "TEST_NOT_FOUND_STR",
        Security("TEST_NOT_FOUND_POS"),
    ]
    expected_map = {
        key_list[0]: Position("TEST_SEC_A", 1000),
        key_list[1]: Position("TEST_SEC_B", 2000),
    }

    # WHEN
    test_obj = implementations.account_solution.Account(
        expected_positions, expected_name
    )
    return_pos_itr = test_obj.get_positions(key_list)

    # EXPECT
    assert len(return_pos_itr) == len(key_list) - 2
    print(return_pos_itr)
    for item in key_list:
        if isinstance(item, Security) and "NOT_FOUND" in item.get_name():
            assert item not in return_pos_itr
        elif isinstance(item, str) and "NOT_FOUND" in item:
            assert item not in return_pos_itr
        else:
            assert item in return_pos_itr
            assert (
                return_pos_itr[item].get_security().get_name()
                == expected_map[item].get_security().get_name()
            )
            assert (
                return_pos_itr[item].get_position()
                == expected_map[item].get_position()
            )


def test_add_positions():
    expected_name = "MY TEST ACCOUNT"
    start_positions = {
        Position("TEST_SEC_A", 1000),
        Position("TEST_SEC_B", 2000),
    }
    update_positions = {
        Position("TEST_SEC_B", 3000),
        Position("TEST_SEC_C", 1500),
    }
    expected_positions = {
        "TEST_SEC_A": 1000,
        "TEST_SEC_B": 3000,
        "TEST_SEC_C": 1500,
    }

    # WHEN
    test_obj = implementations.account_solution.Account(
        start_positions, expected_name
    )
    test_obj.add_positions(update_positions)
    return_pos_itr = test_obj.get_all_positions()

    # EXPECT
    assert len(return_pos_itr) == len(expected_positions)

    for item in list(return_pos_itr):
        assert item.get_security().get_name() in expected_positions
        assert (
            item.get_position()
            == expected_positions[item.get_security().get_name()]
        )
        del expected_positions[item.get_security().get_name()]
        return_pos_itr.remove(item)

    assert len(return_pos_itr) == 0
    assert len(expected_positions) == 0


def test_removePositions():
    expected_name = "MY TEST ACCOUNT"
    start_positions = {
        Position("TEST_SEC_A", 1000),
        Position("TEST_SEC_B", 2000),
        Position("TEST_SEC_C", 1500),
        Position("TEST_SEC_D", 3500),
    }
    remove_positions = {Security("TEST_SEC_B"), Security("TEST_SEC_C")}
    expected_positions = {"TEST_SEC_A": 1000, "TEST_SEC_D": 3500}

    # WHEN
    test_obj = implementations.account_solution.Account(
        start_positions, expected_name
    )
    test_obj.remove_positions(remove_positions)
    return_pos_itr = test_obj.get_all_positions()

    # EXPECT
    assert len(return_pos_itr) == len(expected_positions)

    for item in list(return_pos_itr):
        assert item.get_security().get_name() in expected_positions
        assert (
            item.get_position()
            == expected_positions[item.get_security().get_name()]
        )
        del expected_positions[item.get_security().get_name()]
        return_pos_itr.remove(item)

    assert len(return_pos_itr) == 0
    assert len(expected_positions) == 0